In [1]:
from itertools import combinations
from operator import itemgetter
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
import dateutil.parser
import time

In [2]:
customer_df = pd.read_csv('customerdata.csv',low_memory=False)
# customer_df = pd.read_csv('customerdata.csv',low_memory=False, nrows = 1000)

In [3]:
customer_df

,name,DELIVERY_DATE,CUSTOMER_ID,id,TIME_CREATED,product_item_id,PRODUCT_NAME,CATEGORY,QUANTITY,DELIVERY_QUANTITY,status,COMPLAINT_REASON,UNIT_MEASURE,code,PACKAGING,MRP,UNIT_PRICE,DISCOUNT
0,Bangalore,2021-03-07T00:00:00.000+05:30,24,23181828,2021-03-06T07:15:05.000+05:30,41534,Nandini Shubham-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,22.0,22.0,0
1,Bangalore,2021-03-07T00:00:00.000+05:30,24,23181828,2021-03-06T07:15:05.000+05:30,41540,Nandini Toned Milk-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,19.0,19.0,0
2,Bangalore,2021-03-07T00:00:00.000+05:30,33,23181829,2021-03-06T07:15:05.000+05:30,41569,Heritage Toned Milk-500ML Pouch,Milk,3,3,DELIVERED,NaN,500.0,ML,Pouch,21.0,21.0,0
3,Bangalore,2021-03-07T00:00:00.000+05:30,144,23181830,2021-03-06T07:15:05.000+05:30,41531,Heritage Full Cream Milk-500ML Pouch,Milk,2,2,DELIVERED,NaN,500.0,ML,Pouch,26.0,26.0,0
4,Bangalore,2021-03-07T00:00:00.000+05:30,144,23181830,2021-03-06T07:15:05.000+05:30,41536,Nandini Samrudhi-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,23.0,23.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782404,Pune,2021-03-21T00:00:00.000+05:30,406590,23634170,2021-03-20T19:00:40.000+05:30,79408,Kiwi Green-3PER PIECES Box pack,Fruits,2,1,DELIVERED,NaN,3.0,PER PIECES,Box pack,80.0,48.0,0
782405,Pune,2021-03-21T00:00:00.000+05:30,406590,23634170,2021-03-20T19:00:40.000+05:30,79417,Orange - Nagpur-1KG Net Bag,Fruits,2,2,DELIVERED,NaN,1.0,KG,Net Bag,89.0,70.0,0
782406,Pune,2021-03-21T00:00:00.000+05:30,406590,23634170,2021-03-20T19:00:40.000+05:30,56869,Tender Coconut-1PER PIECE Shrink Wrap,Fruits,1,1,DELIVERED,NaN,1.0,PER PIECE,Shrink Wrap,46.0,0.0,100
782407,Pune,2021-03-21T00:00:00.000+05:30,186941,23634527,2021-03-20T20:30:27.000+05:30,42639,Amul Taaza-500ML Pouch,Milk,1,0,NOT_DELIVERED,SHORTAGE_WAREHOUSE,500.0,ML,Pouch,23.0,23.0,0


In [4]:
customer_df["DELIVERY_DATE"] = pd.to_datetime(customer_df['DELIVERY_DATE'])
customer_df["TIME_CREATED"] = pd.to_datetime(customer_df['TIME_CREATED'])
customer_df.loc[customer_df['name'] == "Bangalore"]

,name,DELIVERY_DATE,CUSTOMER_ID,id,TIME_CREATED,product_item_id,PRODUCT_NAME,CATEGORY,QUANTITY,DELIVERY_QUANTITY,status,COMPLAINT_REASON,UNIT_MEASURE,code,PACKAGING,MRP,UNIT_PRICE,DISCOUNT
0,Bangalore,2021-03-07 00:00:00+05:30,24,23181828,2021-03-06 07:15:05+05:30,41534,Nandini Shubham-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,22.0,22.0,0
1,Bangalore,2021-03-07 00:00:00+05:30,24,23181828,2021-03-06 07:15:05+05:30,41540,Nandini Toned Milk-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,19.0,19.0,0
2,Bangalore,2021-03-07 00:00:00+05:30,33,23181829,2021-03-06 07:15:05+05:30,41569,Heritage Toned Milk-500ML Pouch,Milk,3,3,DELIVERED,NaN,500.0,ML,Pouch,21.0,21.0,0
3,Bangalore,2021-03-07 00:00:00+05:30,144,23181830,2021-03-06 07:15:05+05:30,41531,Heritage Full Cream Milk-500ML Pouch,Milk,2,2,DELIVERED,NaN,500.0,ML,Pouch,26.0,26.0,0
4,Bangalore,2021-03-07 00:00:00+05:30,144,23181830,2021-03-06 07:15:05+05:30,41536,Nandini Samrudhi-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,23.0,23.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779933,Bangalore,2021-03-21 00:00:00+05:30,463337,23634694,2021-03-20 21:30:28+05:30,48762,Britannia Brown Bread-450G Pack,Bread,1,1,DELIVERED,NaN,450.0,G,Pack,45.0,45.0,0
779934,Bangalore,2021-03-21 00:00:00+05:30,463337,23634694,2021-03-20 21:30:28+05:30,79463,Capsicum - Green-500G Net Bag,Vegetables,1,1,DELIVERED,NaN,500.0,G,Net Bag,30.0,23.0,0
779935,Bangalore,2021-03-21 00:00:00+05:30,463337,23634694,2021-03-20 21:30:28+05:30,86806,Ole Moo Farm Fresh Milk-500ML Pouch,Milk,1,0,NOT_DELIVERED,QUALITY,500.0,ML,Pouch,32.0,26.0,0
779936,Bangalore,2021-03-21 00:00:00+05:30,414795,23634758,2021-03-20 22:00:39+05:30,42411,Garlic-100G Net Bag,Vegetables,3,3,DELIVERED,NaN,100.0,G,Net Bag,24.0,24.0,0


In [5]:
mask = (customer_df['DELIVERY_DATE'] > '2021-03-10') & (customer_df['DELIVERY_DATE'] <= '2021-03-11')
customer_df = customer_df.loc[mask]
customer_df

,name,DELIVERY_DATE,CUSTOMER_ID,id,TIME_CREATED,product_item_id,PRODUCT_NAME,CATEGORY,QUANTITY,DELIVERY_QUANTITY,status,COMPLAINT_REASON,UNIT_MEASURE,code,PACKAGING,MRP,UNIT_PRICE,DISCOUNT
203171,Bangalore,2021-03-11 00:00:00+05:30,24,23301364,2021-03-10 07:15:04+05:30,41534,Nandini Shubham-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,22.0,22.0,0
203172,Bangalore,2021-03-11 00:00:00+05:30,24,23301364,2021-03-10 07:15:04+05:30,41540,Nandini Toned Milk-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,19.0,19.0,0
203173,Bangalore,2021-03-11 00:00:00+05:30,33,23301365,2021-03-10 07:15:04+05:30,41569,Heritage Toned Milk-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,21.0,21.0,0
203174,Bangalore,2021-03-11 00:00:00+05:30,144,23301366,2021-03-10 07:15:04+05:30,41531,Heritage Full Cream Milk-500ML Pouch,Milk,2,2,DELIVERED,NaN,500.0,ML,Pouch,26.0,26.0,0
203175,Bangalore,2021-03-11 00:00:00+05:30,144,23301366,2021-03-10 07:15:04+05:30,41536,Nandini Samrudhi-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,23.0,23.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264303,Pune,2021-03-11 00:00:00+05:30,324449,23315874,2021-03-10 07:26:11+05:30,49296,Amul Gold-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,29.0,29.0,0
264304,Pune,2021-03-11 00:00:00+05:30,398267,23319555,2021-03-10 07:29:06+05:30,42639,Amul Taaza-500ML Pouch,Milk,1,0,NOT_DELIVERED,DISPUTE,500.0,ML,Pouch,23.0,23.0,0
264305,Pune,2021-03-11 00:00:00+05:30,398267,23319555,2021-03-10 21:00:53+05:30,42611,Gokul Full Cream Milk-500ML,Milk,2,0,NOT_DELIVERED,DISPUTE,500.0,ML,Pouch,29.0,29.0,0
264306,Pune,2021-03-11 00:00:00+05:30,529856,23328941,2021-03-10 07:36:34+05:30,42639,Amul Taaza-500ML Pouch,Milk,1,1,DELIVERED,NaN,500.0,ML,Pouch,23.0,23.0,0


In [6]:
category = customer_df[['PRODUCT_NAME','CATEGORY']]
category = category.drop_duplicates()
category

,PRODUCT_NAME,CATEGORY
203171,Nandini Shubham-500ML Pouch,Milk
203172,Nandini Toned Milk-500ML Pouch,Milk
203173,Heritage Toned Milk-500ML Pouch,Milk
203174,Heritage Full Cream Milk-500ML Pouch,Milk
203175,Nandini Samrudhi-500ML Pouch,Milk
...,...,...
263742,Tong Garden Wasabi Coated Green Peas-90G Cup,"Dry Fruits, Nuts & Seeds"
263856,Id Fresh Instant Strong Filter Coffee Liquid-1...,Coffee
263901,Yam-500G Net Bag,Vegetables
263989,Thailand Guava-1PIECE Net Bag,Fruits


In [7]:
category.to_csv('category24.csv')

In [8]:
frequent = pd.DataFrame(customer_df.groupby('id')["PRODUCT_NAME"].apply(list))
frequent

,PRODUCT_NAME
id,
23301364,"[Nandini Shubham-500ML Pouch, Nandini Toned Mi..."
23301365,[Heritage Toned Milk-500ML Pouch]
23301366,"[Heritage Full Cream Milk-500ML Pouch, Nandini..."
23301367,"[Nandini Shubham-500ML Pouch, Nandini Curd-500..."
23301368,[Heritage Standardised Milk-500ML Pouch]
...,...
23331980,[Amul Taaza-500ML Pouch]
23331981,[Amul Taaza-500ML Pouch]
23331982,[Amul Taaza-500ML Pouch]


In [9]:
product_category = pd.DataFrame(customer_df.groupby('id')["PRODUCT_NAME"].apply(list))

In [10]:
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [11]:
item_category = to_1D(frequent["PRODUCT_NAME"])
item_category

0                 Nandini Shubham-500ML Pouch
1              Nandini Toned Milk-500ML Pouch
2             Heritage Toned Milk-500ML Pouch
3        Heritage Full Cream Milk-500ML Pouch
4                Nandini Samrudhi-500ML Pouch
                         ...                 
56324                  Amul Taaza-500ML Pouch
56325                  Amul Taaza-500ML Pouch
56326                  Amul Taaza-500ML Pouch
56327              Chitale Full Cream Milk-1L
56328                  Amul Taaza-500ML Pouch
Length: 56329, dtype: object

In [12]:
# item_category.to_frame('PRODUCT_NAME').merge(customer_df, on='PRODUCT_NAME', how='outer')
# item_category.to_frame('PRODUCT_NAME').merge(customer_df, on='PRODUCT_NAME', how='inner')

In [13]:
unique_items = to_1D(frequent["PRODUCT_NAME"]).value_counts()

In [14]:
unique_items

Nandini Toned Milk-500ML Pouch                      5808
Nandini Shubham-500ML Pouch                         4205
Nandini Curd-500ML Pouch                            2717
Nandini Toned Milk-1L Pouch                         2399
Nandini Samrudhi-500ML Pouch                        2091
                                                    ... 
Scotch Brite Scrub Sponge Large-1PIECE Pack            1
Pro Nature Organic Urad Black - Whole-500G Pouch       1
Tropicana Mango-1L Tetra Pack                          1
Popular Essentials Black Cardamom-50G Pouch            1
Naturally Nutritious Raw Peanuts-1KG Pouch             1
Length: 1431, dtype: int64

In [15]:
def boolean_df(item_lists, unique_items):

    # Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask that returns a True-False list of whether a tag is in a taglist
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

In [16]:
frequent_bool = boolean_df(frequent["PRODUCT_NAME"], unique_items.keys())

In [17]:
frequent_bool

,Nandini Toned Milk-500ML Pouch,Nandini Shubham-500ML Pouch,Nandini Curd-500ML Pouch,Nandini Toned Milk-1L Pouch,Nandini Samrudhi-500ML Pouch,Nandini Shubham-1L Pouch,Nandini Cow Milk-500ML Pouch,Tender Coconut-1PER PIECE Shrink Wrap,Heritage Toned Milk-500ML Pouch,Amul Taaza-500ML Pouch,...,Maiyas Bombay Mixture-185G Pouch,Premium Mix-140G Box,Grapes Red Globe-500G MAP Pack,Dabur Hommade Garlic Paste-200G Spout Pouch,MTR Spicy Sambhar Powder-200G Pack,Scotch Brite Scrub Sponge Large-1PIECE Pack,Pro Nature Organic Urad Black - Whole-500G Pouch,Tropicana Mango-1L Tetra Pack,Popular Essentials Black Cardamom-50G Pouch,Naturally Nutritious Raw Peanuts-1KG Pouch
id,,,,,,,,,,,,,,,,,,,,,
23301364,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
23301365,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
23301366,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
23301367,False,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
23301368,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23331980,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
23331981,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
23331982,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [18]:
tic = time.time()
frequent_int = frequent_bool.astype(int)
frequent_int.head(10)
toc = time.time()
toc - tic

0.09474372863769531

In [19]:
tic = time.time()

frequent_freq_mat = np.dot(frequent_int.T, frequent_int)

toc = time.time()
toc - tic

47.786519289016724

In [20]:
tic = time.time()

frequent_freq = pd.DataFrame(frequent_freq_mat, columns = unique_items.keys(), index = unique_items.keys())

toc = time.time()
toc - tic

0.0

In [21]:
frequent_freq

,Nandini Toned Milk-500ML Pouch,Nandini Shubham-500ML Pouch,Nandini Curd-500ML Pouch,Nandini Toned Milk-1L Pouch,Nandini Samrudhi-500ML Pouch,Nandini Shubham-1L Pouch,Nandini Cow Milk-500ML Pouch,Tender Coconut-1PER PIECE Shrink Wrap,Heritage Toned Milk-500ML Pouch,Amul Taaza-500ML Pouch,...,Maiyas Bombay Mixture-185G Pouch,Premium Mix-140G Box,Grapes Red Globe-500G MAP Pack,Dabur Hommade Garlic Paste-200G Spout Pouch,MTR Spicy Sambhar Powder-200G Pack,Scotch Brite Scrub Sponge Large-1PIECE Pack,Pro Nature Organic Urad Black - Whole-500G Pouch,Tropicana Mango-1L Tetra Pack,Popular Essentials Black Cardamom-50G Pouch,Naturally Nutritious Raw Peanuts-1KG Pouch
Nandini Toned Milk-500ML Pouch,5807,389,880,603,176,87,91,184,37,0,...,0,0,0,0,1,0,0,1,0,0
Nandini Shubham-500ML Pouch,389,4205,571,77,62,404,66,105,32,0,...,0,0,0,0,0,0,0,0,0,0
Nandini Curd-500ML Pouch,880,571,2717,312,233,201,245,90,49,0,...,0,0,0,0,1,0,0,1,0,0
Nandini Toned Milk-1L Pouch,603,77,312,2399,41,92,34,64,7,0,...,0,0,0,0,0,0,0,0,0,0
Nandini Samrudhi-500ML Pouch,176,62,233,41,2091,24,59,53,13,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Scotch Brite Scrub Sponge Large-1PIECE Pack,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Pro Nature Organic Urad Black - Whole-500G Pouch,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Tropicana Mango-1L Tetra Pack,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Popular Essentials Black Cardamom-50G Pouch,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [22]:
frequent_freq.to_csv('frequent_freq24.csv')

In [23]:
tic = time.time()

customer_df = pd.read_csv('customerdata.csv',low_memory=False)
customer_df["DELIVERY_DATE"] = pd.to_datetime(customer_df['DELIVERY_DATE'])
customer_df["TIME_CREATED"] = pd.to_datetime(customer_df['TIME_CREATED'])
customer_df.loc[customer_df['name'] == "Bangalore"]
mask = (customer_df['DELIVERY_DATE'] > '2021-03-10') & (customer_df['DELIVERY_DATE'] <= '2021-03-11')
customer_df = customer_df.loc[mask]
category = customer_df[['PRODUCT_NAME','CATEGORY']]
category = category.drop_duplicates()
category.to_csv('category24.csv')
frequent = pd.DataFrame(customer_df.groupby('id')["PRODUCT_NAME"].apply(list))
unique_items = to_1D(frequent["PRODUCT_NAME"]).value_counts()
frequent_bool = boolean_df(frequent["PRODUCT_NAME"], unique_items.keys())
frequent_int = frequent_bool.astype(int)
frequent_freq_mat = np.dot(frequent_int.T, frequent_int)
frequent_freq = pd.DataFrame(frequent_freq_mat, columns = unique_items.keys(), index = unique_items.keys())
frequent_freq.to_csv('frequent_freq24.csv')

toc = time.time()
toc - tic

60.51228880882263